# Treinamento de Modelo para Detecção de Fake News

Este notebook realiza o treinamento dos modelos de machine learning para detecção de fake news e exporta os modelos treinados para o sistema de armazenamento S3.